In [6]:
import osqp
import numpy as np
import scipy as sp
from scipy import sparse
import utils
from params import *

In [7]:
def calculate_Rz(euler):
    """_summary_

    Args:
        euler (_type_): _description_
    """
    cos_yaw = np.cos(euler[2])
    sin_yaw = np.sin(euler[2])
    cos_pitch = np.cos(euler[1])
    tan_pitch = np.tan(euler[1])
            
    Rz = np.array([
        [cos_yaw / cos_pitch, sin_yaw / cos_pitch, 0],
        [-sin_yaw, cos_yaw, 0],
        [cos_yaw * tan_pitch, sin_yaw * tan_pitch, 1]
    ]) #Transpose of Rz
    return Rz
    
def calculate_A_mat(Rz):
    """_summary_

    Args:
        Rz (_type_): _description_

    Returns:
        _type_: _description_
    """
    O3 = np.zeros((3,3))
    I3 = np.eye(3)
    
    A_mat = np.block([
        [O3,O3,Rz,O3],
        [O3,O3,O3,I3],
        [O3,O3,O3,O3],
        [O3,O3,O3,O3]
    ])
    return A_mat
    
def calculate_B_mat(robot_mass, trunk_inertia, Rz,leg_pose):
    """_summary_

    Args:
        robot_mass (_type_): _description_
        trunk_inertia (_type_): _description_
        Rz (_type_): _description_
        leg_pose (_type_): _description_

    Returns:
        _type_: _description_
    """
    I = Rz.T@trunk_inertia@Rz
    row = []
    for i in range(4): #for each leg
        O3 = np.zeros((3,3))
        I3 = np.eye(3)
        row.append(np.array([
            [O3],
            [O3],
            [np.linalg.inv(I)@utils.skew(leg_pose[i])],
            [I3/robot_mass]
        ]))
    B_mat = np.block([row[1],row[2],row[3],row[4]])
    return B_mat

In [8]:
# Constants #

M = ROBOT_MASS
g = 9.81

Ts = 50e-3

I = np.array([
    [1.68399345e-02, 8.39020140e-05, 5.97678823e-04],
    [8.39020140e-05, 5.65790460e-02, 2.51339807e-05],
    [5.97678823e-04, 2.51339807e-05, 6.47136047e-02]
])

leg_pose = np.array([
    [1.68399345e-02, 8.39020140e-05, 0.],
    [8.39020140e-05, 5.65790460e-02, 0.],
    [8.39020140e-05, 5.65790460e-02, 0.],
    [5.97678823e-04, 2.51339807e-05, 0.]
])

In [11]:
euler = np.zeros(3)    
Rz = calculate_Rz(euler)
Ad =calculate_A_mat(Rz)
Ad = sparse.csc_matrix(Ad)

Bd =calculate_A_mat(M, I, Rz,leg_pose)
Bd = sparse.csc_matrix(Bd)
Bd = sparse.csc_matrix([
  [0.,      -0.0726,  0.,     0.0726],
  [-0.0726,  0.,      0.0726, 0.    ],
  [-0.0152,  0.0152, -0.0152, 0.0152],
  [-0.,     -0.0006, -0.,     0.0006],
  [0.0006,   0.,     -0.0006, 0.0000],
  [0.0106,   0.0106,  0.0106, 0.0106],
  [0,       -1.4512,  0.,     1.4512],
  [-1.4512,  0.,      1.4512, 0.    ],
  [-0.3049,  0.3049, -0.3049, 0.3049],
  [-0.,     -0.0236,  0.,     0.0236],
  [0.0236,   0.,     -0.0236, 0.    ],
  [0.2107,   0.2107,  0.2107, 0.2107]])
[nx, nu] = Bd.shape

In [12]:
# Constraints
umin = 10*np.ones(12)
umax = 666*np.ones(12)
xmin = np.array([-np.pi/6,-np.pi/6,-np.inf,-np.inf,-np.inf,-1.,
                 -np.inf,-np.inf,-np.inf,-np.inf,-np.inf,-np.inf])
xmax = np.array([ np.pi/6, np.pi/6, np.inf, np.inf, np.inf, np.inf,
                  np.inf, np.inf, np.inf, np.inf, np.inf, np.inf])

In [8]:
# Objective function
Q = sparse.diags([1., 1., 1., 0., 0., 50., 0., 0., 1., 1., 1., 1.])
QN = Q
R = 1e-6*sparse.eye(12)

In [9]:
# Initial and reference states
x0 = np.zeros(12)
xr = np.array([0.,0.,0.,0.,0.,0.278,0.,0.,0.,0.,0.,0.])

In [10]:
# Prediction horizon
N = PLAN_HORIZON

# Cast MPC problem to a QP: x = (x(0),x(1),...,x(N),u(0),...,u(N-1))
# - quadratic objective
P = sparse.block_diag([sparse.kron(sparse.eye(N), Q), QN,
                       sparse.kron(sparse.eye(N), R)], format='csc')
# - linear objective
q = np.hstack([np.kron(np.ones(N), -Q.dot(xr)), -QN.dot(xr),
               np.zeros(N*nu)])
# - linear dynamics
Ax = sparse.kron(sparse.eye(N+1),-sparse.eye(nx)) + sparse.kron(sparse.eye(N+1, k=-1), Ad)
Bu = sparse.kron(sparse.vstack([sparse.csc_matrix((1, N)), sparse.eye(N)]), Bd)
Aeq = sparse.hstack([Ax, Bu])
leq = np.hstack([-x0, np.zeros(N*nx)])
ueq = leq
# - input and state constraints
Aineq = sparse.eye((N+1)*nx + N*nu)
lineq = np.hstack([np.kron(np.ones(N+1), xmin), np.kron(np.ones(N), umin)])
uineq = np.hstack([np.kron(np.ones(N+1), xmax), np.kron(np.ones(N), umax)])
# - OSQP constraints
A = sparse.vstack([Aeq, Aineq], format='csc')
l = np.hstack([leq, lineq])
u = np.hstack([ueq, uineq])

In [11]:
# Create an OSQP object
prob = osqp.OSQP()

In [12]:
# Setup workspace
prob.setup(P, q, A, l, u, warm_start=True)

-----------------------------------------------------------------
           OSQP v0.6.2  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 172, constraints m = 304
          nnz(P) + nnz(A) = 1161
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off



In [14]:
# Solve
res = prob.solve()

iter   objective    pri res    dua res    rho        time
   1  -5.4995e+01   1.74e-03   2.41e+00   1.00e-01   7.75e-05s
  25  -5.4996e+01   4.88e-07   9.93e-06   1.00e-01   3.92e-04s

status:               solved
number of iterations: 25
optimal objective:    -54.9961
run time:             4.48e-04s
optimal rho estimate: 4.98e-02



In [13]:
# Simulate in closed loop
nsim = 15
for i in range(nsim):
    # Solve
    res = prob.solve()

    # Check solver status
    if res.info.status != 'solved':
        raise ValueError('OSQP did not solve the problem!')

    # Apply first control input to the plant
    ctrl = res.x[-N*nu:-(N-1)*nu]
    x0 = Ad.dot(x0) + Bd.dot(ctrl)

    # Update initial state
    l[:nx] = -x0
    u[:nx] = -x0
    prob.update(l=l, u=u)

iter   objective    pri res    dua res    rho        time
   1  -3.2965e+01   8.15e-01   6.00e+00   1.00e-01   1.80e-03s
  25  -4.0983e+01   6.50e-05   2.15e-04   1.00e-01   3.86e-03s

status:               solved
number of iterations: 25
optimal objective:    -40.9834
run time:             4.33e-03s
optimal rho estimate: 9.40e-02

iter   objective    pri res    dua res    rho        time
   1  -4.0983e+01   1.67e+00   9.40e+02   1.00e-01   8.97e-05s
  25  -4.6383e+01   4.50e-04   4.45e-03   1.00e-01   3.91e-04s

status:               solved
number of iterations: 25
optimal objective:    -46.3833
run time:             4.58e-04s
optimal rho estimate: 4.51e-02

iter   objective    pri res    dua res    rho        time
   1  -4.6383e+01   9.59e-01   5.39e+02   1.00e-01   6.67e-05s
  25  -5.0969e+01   2.47e-04   2.56e-03   1.00e-01   4.15e-04s

status:               solved
number of iterations: 25
optimal objective:    -50.9693
run time:             6.42e-03s
optimal rho estimate: 4.41e-02